# Write Data to Database

In [1]:
import pandas as pd
import numpy as np

## Connect to Database

In [2]:
import mysql.connector

In [3]:
endpoint = "seratestdatabase.c4cjk1vto1om.us-east-2.rds.amazonaws.com"
port = "3306"
usr = "admin"
pswd = "seracapstone"
region = "us-east-2b"
dbname = "teachsim"

In [4]:
cnx = mysql.connector.connect(user=usr, password=pswd, host=endpoint, database=dbname)

In [5]:
cursor = cnx.cursor(buffered=True)

## Test Connection

In [35]:
cursor.execute("SELECT * FROM Identifiers WHERE id_participant LIKE 'testID%'")
result = cursor.fetchall()
colnames = [x[0] for x in cursor.description]
pd.DataFrame(result, columns=colnames)

,br_treat_cond,br_treat_cond_mr,br_treat_cond_iat,fb_treat_cond,f17_treat_cond,s18_treat_cond,fb_treat_cond_iat,id_participant,id_section,id_site,id_study,id_year


## Set Working Directory

In [6]:
import os
os.getcwd()

'/Users/jmachita03/Documents/GitHub/MSDS_SERA_capstone/RDS'

In [7]:
os.chdir("/Users/jmachita03/Desktop/Capstone/RDS Set Up")

In [77]:
os.chdir('/Users/jmachita03/Documents/GitHub/MSDS_SERA_capstone/RDS')

## Functions

In [8]:
def getColumnNames(tableName, cursor):
    query = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'" + tableName + "'"
    cursor.execute(query)
    result = cursor.fetchall()
    result2 = [x[0] for x in result]
    
    return result2

In [9]:
def overlapColumns(cols1, cols2):
    overlap = []
    for col in cols1:
        if col in cols2:
            overlap = overlap + [col]
    return overlap

In [10]:
def replaceQueries(df, columns, tableName):
    queries = []
    for index, row in df.iterrows():
        query = "REPLACE INTO " +  tableName + " "
        cols = "("
        vals = "("
        for col in columns:
            val = row[col]
            if type(val) == float or type(val) == int:
                if np.isnan(val) == True:
                    continue
            if type(val) == str:
                vals = vals + "'" + val + "', "
            else: vals = vals + str(val) + ", "
            
            cols = cols + str(col) + ", "

        query = query + cols[:-2] + ") " + 'VALUES ' + vals[:-2] + ");"
        queries = queries + [query]
    return queries

In [11]:
def updateQueries(df, columns, tableName):
    queries = []
    for index, row in df.iterrows():
        req_update = False
        query = "UPDATE " + tableName + " SET "
        for col in columns:
            newval = row[col]
            if newval != None:
                query = query + col + " = " + str(newval) + " , "
                req_update = True
        if req_update == True:
            query = query[:-2]
            query = query + "WHERE id_participant = '" + row["id_participant"] + "';"
            queries = queries + [query]
    return queries

In [12]:
def getExistingIds(tableName, cursor):
    query = "SELECT id_participant FROM " + str(tableName)
    cursor.execute(query)
    result = cursor.fetchall()
    existing_ids = [x[0] for x in result]
    return existing_ids

In [13]:
# encompasses a combo of the replaceQueries and updateQueries functions
def list_queries(df, columns, tableName, existing_ids):
    queries = []
    for index, row in df.iterrows():
        # if the row for that participant already exists, just update it
        if row['id_participant'] in existing_ids:
            req_update = False
            query = "UPDATE " + tableName + " SET "
            for col in columns:
                newval = row[col]
                if type(newval) == str:
                    query = query + col + " = '" + newval + "' , "
                    req_update = True
                elif np.isnan(float(newval)) == False: 
                    query = query + col + " = " + str(newval) + " , "
                    req_update = True
            if req_update == True:
                query = query[:-2]
                query = query + "WHERE id_participant = '" + row["id_participant"] + "';"
                queries = queries + [query]
        
        # if that participant is not in the database, create/replace the row
        else:
            req_update = False
            query = "REPLACE INTO " +  tableName + " "
            cols = "("
            vals = "("
            for col in columns:
                val = row[col] 
                if type(val) == str:
                    cols = cols + str(col) + ", "
                    vals = vals + "'" + val + "', "
                    req_update = True
                elif np.isnan(float(val)) == False: 
                    cols = cols + str(col) + ", "
                    vals = vals + str(val) + ", "
                    req_update = True
            query = query + cols[:-2] + ") " + 'VALUES ' + vals[:-2] + ");"
            if req_update == True:
                queries = queries + [query]
                existing_ids = existing_ids + [row['id_participant']]
            
    return queries

In [14]:
def generateQueries(df, tableName, cursor):
    df_cols = df.columns
    tab_cols = getColumnNames(tableName, cursor)
    overlap = overlapColumns(df_cols, tab_cols)
    
    existing_ids = getExistingIds(tableName, cursor)
    
    query_list = list_queries(df, overlap, tableName, existing_ids)
    
    return query_list

In [15]:
pd.__file__

'/Users/jmachita03/opt/anaconda3/lib/python3.7/site-packages/pandas/__init__.py'

## Get Id_Participants Already in Table

In [27]:
cursor.execute("SELECT id_participant FROM Identifiers")
result = cursor.fetchall()
existing_ids = [x[0] for x in result]
# existing_ids

## Import Cleaned Dataset

In [8]:
data = pd.read_csv("outcome_merged.csv")

In [9]:
cleaned_cols = list(data.columns)

In [10]:
data2 = data.dropna(axis='rows', subset=['id_participant'])

In [37]:
uploaded_data = data2

## Test generateQueries function

In [32]:
quer = generateQueries(data2, "Identifiers", cursor)

In [33]:
quer

["UPDATE Identifiers SET fb_treat_cond = 0.0 , id_participant = '1_1819_2_67' , id_section = 2.0 , id_site = 1.0 , id_year = 1819.0 WHERE id_participant = '1_1819_2_67';",
 "UPDATE Identifiers SET fb_treat_cond = 0.0 , id_participant = '1_1819_2_67' , id_section = 2.0 , id_site = 1.0 , id_year = 1819.0 WHERE id_participant = '1_1819_2_67';",
 "UPDATE Identifiers SET fb_treat_cond = 0.0 , id_participant = '1_1819_3_86' , id_section = 3.0 , id_site = 1.0 , id_year = 1819.0 WHERE id_participant = '1_1819_3_86';",
 "UPDATE Identifiers SET fb_treat_cond = 0.0 , id_participant = '1_1819_3_86' , id_section = 3.0 , id_site = 1.0 , id_year = 1819.0 WHERE id_participant = '1_1819_3_86';",
 "UPDATE Identifiers SET fb_treat_cond = 0.0 , id_participant = '1_1819_3_86' , id_section = 3.0 , id_site = 1.0 , id_year = 1819.0 WHERE id_participant = '1_1819_3_86';",
 "UPDATE Identifiers SET fb_treat_cond = 0.0 , id_participant = '1_1819_4_108' , id_section = 4.0 , id_site = 1.0 , id_year = 1819.0 WHERE i

## Test Loop of Queries

In [89]:
tables = ['Identifiers', 'Survey_Measures', 'Participant_Measures', 'Performance_Measures']
for table in tables:
    queries = generateQueries(uploaded_data, table, cursor)
    for query in queries:
        cursor.execute(query)

## Test getQueries Function

In [32]:
existing_ids = existing_ids[:50]

In [40]:
queries = getQueries(data2, overlap, 'Identifiers', existing_ids)
queries

["REPLACE INTO Identifiers (fb_treat_cond, id_participant, id_section, id_site, id_year) VALUES (0.0, '1_1819_2_67', 2.0, 1.0, 1819.0);",
 "UPDATE Identifiers SET fb_treat_cond = 0.0 , id_participant = '1_1819_2_67' , id_section = 2.0 , id_site = 1.0 , id_year = 1819.0 WHERE id_participant = '1_1819_2_67';",
 "REPLACE INTO Identifiers (fb_treat_cond, id_participant, id_section, id_site, id_year) VALUES (0.0, '1_1819_3_86', 3.0, 1.0, 1819.0);",
 "UPDATE Identifiers SET fb_treat_cond = 0.0 , id_participant = '1_1819_3_86' , id_section = 3.0 , id_site = 1.0 , id_year = 1819.0 WHERE id_participant = '1_1819_3_86';",
 "UPDATE Identifiers SET fb_treat_cond = 0.0 , id_participant = '1_1819_3_86' , id_section = 3.0 , id_site = 1.0 , id_year = 1819.0 WHERE id_participant = '1_1819_3_86';",
 "REPLACE INTO Identifiers (fb_treat_cond, id_participant, id_section, id_site, id_year) VALUES (0.0, '1_1819_4_108', 4.0, 1.0, 1819.0);",
 "UPDATE Identifiers SET fb_treat_cond = 0.0 , id_participant = '1_18

In [41]:
for query in queries:
    cursor.execute(query)

## Simplified Way

In [86]:
queries = generateQueries(uploaded_data, "Performance_Measures", cursor)

In [87]:
queries

["UPDATE Performance_Measures SET id_participant = '1_1819_2_67' WHERE id_participant = '1_1819_2_67';",
 "UPDATE Performance_Measures SET id_participant = '1_1819_2_67' WHERE id_participant = '1_1819_2_67';",
 "UPDATE Performance_Measures SET id_participant = '1_1819_3_86' WHERE id_participant = '1_1819_3_86';",
 "UPDATE Performance_Measures SET id_participant = '1_1819_3_86' WHERE id_participant = '1_1819_3_86';",
 "UPDATE Performance_Measures SET id_participant = '1_1819_3_86' WHERE id_participant = '1_1819_3_86';",
 "UPDATE Performance_Measures SET id_participant = '1_1819_4_108' WHERE id_participant = '1_1819_4_108';",
 "UPDATE Performance_Measures SET id_participant = '1_1819_4_108' WHERE id_participant = '1_1819_4_108';",
 "UPDATE Performance_Measures SET id_participant = '1_1819_4_108' WHERE id_participant = '1_1819_4_108';",
 "UPDATE Performance_Measures SET id_participant = '1_1819_2_76' WHERE id_participant = '1_1819_2_76';",
 "UPDATE Performance_Measures SET id_participant 

In [88]:
for query in queries:
    cursor.execute(query)

## Identifiers Table

In [16]:
id_cols = getColumnNames("Identifiers")

In [17]:
overlap = overlapColumns(cleaned_cols, id_cols)

In [17]:
queries = replaceQueries(data2, overlap, 'Identifiers')

In [56]:
for query in queries:
    cursor.execute(query)

In [28]:
# view table
cursor.execute("SELECT * FROM Identifiers")
result = cursor.fetchall()
colnames = [x[0] for x in cursor.description]
pd.DataFrame(result, columns=colnames)

,br_treat_cond,br_treat_cond_mr,br_treat_cond_iat,fb_treat_cond,f17_treat_cond,s18_treat_cond,fb_treat_cond_iat,id_participant,id_section,id_site,id_study,id_year
0,None,None,None,0,None,None,None,1_1819_1_1,1,1.0,None,1819
1,None,None,None,0,None,None,None,1_1819_1_10,1,1.0,None,1819
2,None,None,None,1,None,None,None,1_1819_1_13,1,1.0,None,1819
3,None,None,None,1,None,None,None,1_1819_1_14,1,1.0,None,1819
4,None,None,None,0,None,None,None,1_1819_1_16,1,1.0,None,1819
...,...,...,...,...,...,...,...,...,...,...,...,...
100,None,None,None,0,None,None,None,1_1819_4_114,4,1.0,None,1819
101,None,None,None,1,None,None,None,1_1819_4_115,4,1.0,None,1819
102,None,None,None,0,None,None,None,1_1819_4_116,4,1.0,None,1819
103,None,None,None,1,None,None,None,1_1819_4_118,4,1.0,None,1819


In [10]:
test_d = {'id_participant':['testID1', 'testID2'], 'id_section':[3,2], 'id_site':[1,1], 'id_year':[2021, 1819], 'fb_treat_cond':[1,0], 's18_treat_cond':[3,2]}



In [11]:
test_data = pd.DataFrame(test_data)

In [13]:
test_data.to_csv("test_data.csv")

## Participant Measures

In [55]:
part_cols = getColumnNames("Participant_Measures")

In [56]:
overlap = overlapColumns(cleaned_cols, part_cols)

In [59]:
queries = replaceQueries(data2, overlap, 'Participant_Measures')

In [60]:
queries

["REPLACE INTO Participant_Measures (age, ccs_cohort, ccs_gpa, ccs_major, ccs_majother, ccs_stutype, crtse_05, crtse_06, crtse_07, crtse_08, crtse_09, crtse_10, crtse_12, crtse_13, crtse_14, crtse_15, crtse_16, crtse_17, crtse_18, crtse_19, crtse_20, crtse_22, crtse_23, crtse_24, crtse_25, crtse_27, crtse_28, crtse_30, crtse_31, crtse_33, crtse_35, crtse_36, crtse_41, crtse_total, das_01, das_02, das_03, das_04, das_05, das_06, das_07, das_08, das_09, das_10, das_11, das_12, das_13, das_14, das_15, das_16, das_17, das_18, das_19, das_20, das_21, das_anxiety, das_depression, das_stress, faedu, fit_01, fit_04, fit_05, fit_06, fit_07, fit_08, fit_09, fit_10, fit_11, fit_12, fit_13, fit_total, fustuach, fusturace, fustuses, futeaach, futearace, futeases, gender, grit_01, grit_04, grit_06, grit_09, grit_10, grit_12, grit_total, holang, holangother, hsach, hsloc, hsrace, hsses, hstype, id_participant, imts_01, imts_02, imts_03, imts_04, imts_05, imts_total, moedu, neo_02, neo_04, neo_05, neo

In [61]:
for query in queries:
    cursor.execute(query)

In [62]:
# view table
cursor.execute("SELECT * FROM Participant_Measures")
result = cursor.fetchall()
colnames = [x[0] for x in cursor.description]
pd.DataFrame(result, columns=colnames)

,id_participant,ccs_cohort,ccs_stutype,ccs_major,ccs_majother,ccs_gpa,partch,moedu,faedu,gender,...,crtse_25,crtse_27,crtse_28,crtse_30,crtse_31,crtse_33,crtse_35,crtse_36,crtse_41,crtse_total
0,1_1819_1_1,2017.0,2.0,10.0,0.0,3.455,2.0,4.0,5.0,2.0,...,70.0,70.0,90.0,90.0,70.0,90.0,70.0,70.0,75.0,70.0
1,1_1819_1_10,2017.0,2.0,10.0,0.0,2.8,4.0,4.0,3.0,2.0,...,20.0,25.0,25.0,40.0,10.0,50.0,36.0,75.0,55.0,38.0
2,1_1819_1_13,2017.0,2.0,10.0,0.0,3.7,4.0,4.0,3.0,1.0,...,23.0,48.0,49.0,32.0,36.0,61.0,58.0,56.0,65.0,55.0
3,1_1819_1_14,2017.0,2.0,1.0,0.0,3.8,3.0,5.0,5.0,1.0,...,98.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
4,1_1819_1_16,2017.0,2.0,8.0,0.0,3.6,4.0,4.0,3.0,2.0,...,56.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,1_1819_4_114,2018.0,1.0,3.0,0.0,3.513,4.0,4.0,3.0,1.0,...,85.0,70.0,80.0,60.0,65.0,75.0,75.0,85.0,80.0,73.0
101,1_1819_4_115,2018.0,1.0,8.0,0.0,3.635,4.0,3.0,3.0,1.0,...,80.0,55.0,80.0,85.0,55.0,70.0,65.0,80.0,60.0,69.0
102,1_1819_4_116,2018.0,1.0,10.0,0.0,3.8,4.0,4.0,5.0,2.0,...,10.0,99.0,100.0,5.0,5.0,99.0,30.0,50.0,85.0,62.0
103,1_1819_4_118,2018.0,1.0,10.0,0.0,2.88,1.0,5.0,4.0,2.0,...,72.0,62.0,67.0,54.0,45.0,73.0,50.0,56.0,76.0,62.0


## Survey Measures

In [63]:
surv_cols = getColumnNames("Survey_Measures")

In [64]:
overlap = overlapColumns(cleaned_cols, surv_cols)

In [65]:
queries = replaceQueries(data2, overlap, 'Survey_Measures')

In [66]:
queries

["REPLACE INTO Survey_Measures (id_participant) VALUES ('1_1819_2_67');",
 "REPLACE INTO Survey_Measures (id_participant) VALUES ('1_1819_2_67');",
 "REPLACE INTO Survey_Measures (id_participant) VALUES ('1_1819_3_86');",
 "REPLACE INTO Survey_Measures (id_participant) VALUES ('1_1819_3_86');",
 "REPLACE INTO Survey_Measures (id_participant) VALUES ('1_1819_3_86');",
 "REPLACE INTO Survey_Measures (id_participant) VALUES ('1_1819_4_108');",
 "REPLACE INTO Survey_Measures (id_participant) VALUES ('1_1819_4_108');",
 "REPLACE INTO Survey_Measures (id_participant) VALUES ('1_1819_4_108');",
 "REPLACE INTO Survey_Measures (id_participant) VALUES ('1_1819_2_76');",
 "REPLACE INTO Survey_Measures (id_participant) VALUES ('1_1819_2_76');",
 "REPLACE INTO Survey_Measures (id_participant) VALUES ('1_1819_2_76');",
 "REPLACE INTO Survey_Measures (id_participant) VALUES ('1_1819_1_14');",
 "REPLACE INTO Survey_Measures (id_participant) VALUES ('1_1819_1_14');",
 "REPLACE INTO Survey_Measures (id_

In [67]:
for query in queries:
    cursor.execute(query)

In [68]:
# view table
cursor.execute("SELECT * FROM Survey_Measures")
result = cursor.fetchall()
colnames = [x[0] for x in cursor.description]
pd.DataFrame(result, columns=colnames)

,id_participant,base_app_dev_adjust,base_app_dev_challenge,base_app_dev_coach,base_app_dev_counselor,base_app_dev_discuss,base_app_dev_parconf,base_app_dev_plan,base_app_dev_referral,base_app_dev_request,...,haber_explain_stud_suc,haber_explain_teach_suc,haber_fallibility,haber_org_and_plan,haber_persistence,haber_survive_in_bureaucracy,haber_testdate,haber_theory_to_practice,haber_total,haber_values_students_learning
0,1_1819_1_1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,1_1819_1_10,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,1_1819_1_13,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,1_1819_1_14,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,1_1819_1_16,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,1_1819_4_114,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
101,1_1819_4_115,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
102,1_1819_4_116,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
103,1_1819_4_118,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


## Performance Measures

In [92]:
perf_cols = getColumnNames("Performance_Measures")

In [93]:
overlap = overlapColumns(cleaned_cols, perf_cols)

In [94]:
queries = replaceQueries(data2, overlap, 'Performance_Measures')

In [95]:
queries

["REPLACE INTO Performance_Measures (id_participant) VALUES ('1_1819_2_67');",
 "REPLACE INTO Performance_Measures (id_participant) VALUES ('1_1819_2_67');",
 "REPLACE INTO Performance_Measures (id_participant) VALUES ('1_1819_3_86');",
 "REPLACE INTO Performance_Measures (id_participant) VALUES ('1_1819_3_86');",
 "REPLACE INTO Performance_Measures (id_participant) VALUES ('1_1819_3_86');",
 "REPLACE INTO Performance_Measures (id_participant) VALUES ('1_1819_4_108');",
 "REPLACE INTO Performance_Measures (id_participant) VALUES ('1_1819_4_108');",
 "REPLACE INTO Performance_Measures (id_participant) VALUES ('1_1819_4_108');",
 "REPLACE INTO Performance_Measures (id_participant) VALUES ('1_1819_2_76');",
 "REPLACE INTO Performance_Measures (id_participant) VALUES ('1_1819_2_76');",
 "REPLACE INTO Performance_Measures (id_participant) VALUES ('1_1819_2_76');",
 "REPLACE INTO Performance_Measures (id_participant) VALUES ('1_1819_1_14');",
 "REPLACE INTO Performance_Measures (id_participa

In [96]:
for query in queries:
    cursor.execute(query)

In [97]:
# view table
cursor.execute("SELECT * FROM Performance_Measures")
result = cursor.fetchall()
colnames = [x[0] for x in cursor.description]
pd.DataFrame(result, columns=colnames)

,id_participant,br_base_affect,br_base_b1ac,br_base_b1ac_agree,br_base_b1cu,br_base_b1cu_agree,br_base_b1oc,br_base_b1oc_agree,br_base_b1re,br_base_b1re_agree,...,fb_pre_f_rest_agree,fb_pre_f_txt,fb_pre_f_txt_agree,fb_pre_quality,fb_pre_quality_agree,fb_pre_tot_hit,fb_pre_tot_hit_agree,fb_pre_tot_oc,fb_pre_tot_oc_agree,fb_pre_vid
0,1_1819_1_1,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,1_1819_1_10,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,1_1819_1_13,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,1_1819_1_14,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,1_1819_1_16,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,1_1819_4_114,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
101,1_1819_4_115,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
102,1_1819_4_116,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
103,1_1819_4_118,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


## Playing Around with Schema

In [35]:
cursor.execute("SELECT column_name, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'Identifiers'")

In [36]:
cursor.fetchall()

[('br_treat_cond', b'int'),
 ('br_treat_cond_mr', b'int'),
 ('br_treat_cond_iat', b'int'),
 ('fb_treat_cond', b'int'),
 ('f17_treat_cond', b'int'),
 ('s18_treat_cond', b'int'),
 ('fb_treat_cond_iat', b'int'),
 ('id_participant', b'varchar'),
 ('id_section', b'int'),
 ('id_site', b'varchar'),
 ('id_study', b'int'),
 ('id_year', b'int')]

## Close Database Connection

In [98]:
cnx.commit()
cnx.close()